In [ ]:
import json
import random
from string import Template
import numpy as np
import PIL
from ipywidgets import widgets, HBox
from random import randrange
import sys
import cv2
import numpy as np
from random import shuffle

In [ ]:
path = "malevic/data/pos1/"

In [ ]:
with open(path + "annotation/train_annotation.json") as f1:
    annotation = json.load(f1)

In [64]:
obj_color_size = {}
# separate shapes by color and size
for img_id, descr in annotation.items():
    for obj in descr[1]['objects']:
        counter = 0
        for another_obj in descr[1]['objects']:
            if (another_obj['color'] == obj['color'] and another_obj['shape'] == obj['shape']):
                counter += 1
        if counter < 2:
            key = (obj['shape'], obj['color'], obj['size'])
            if key in obj_color_size:
                obj_color_size[key].append((img_id, obj['object_id'], obj['size']))
            else:
                obj_color_size[key] = [(img_id, obj['object_id'], obj['size'])]

objects = obj_color_size.copy()

In [66]:
# calculate number of small and big objects on the pictures

counter = 0
smol = 0
big = 0
for img_id, descr in annotation.items():
    for obj in descr[1]['objects']:
        if obj['size'] == 'big':
            big += 1
        else:
            smol += 1
print(smol, big)

66039 26405


In [67]:
shapes = ('triangle', 'circle', 'square', 'rectangle')
sizes = ('big', 'small')
colors = ('red', 'blue', 'yellow', 'green', 'white')

In [68]:
for key, item in objects.items():
    print(key,  ':', len(item))
len(objects)

('circle', 'red', 'big') : 545
('circle', 'green', 'big') : 550
('circle', 'white', 'small') : 1180
('circle', 'blue', 'small') : 1256
('circle', 'red', 'small') : 1246
('circle', 'yellow', 'small') : 1291
('triangle', 'red', 'small') : 1272
('triangle', 'white', 'small') : 1268
('triangle', 'yellow', 'small') : 1268
('triangle', 'green', 'small') : 1274
('triangle', 'white', 'big') : 556
('triangle', 'yellow', 'big') : 592
('triangle', 'blue', 'small') : 1244
('triangle', 'blue', 'big') : 537
('rectangle', 'blue', 'big') : 564
('circle', 'green', 'small') : 1217
('triangle', 'red', 'big') : 528
('square', 'red', 'small') : 1256
('square', 'blue', 'big') : 561
('triangle', 'green', 'big') : 560
('circle', 'blue', 'big') : 542
('rectangle', 'yellow', 'small') : 1285
('rectangle', 'white', 'small') : 1275
('rectangle', 'red', 'small') : 1287
('rectangle', 'green', 'small') : 1300
('square', 'white', 'big') : 516
('square', 'blue', 'small') : 1195
('square', 'green', 'small') : 1242
('squ

40

# Balance objects by shape, color, size

In [69]:
min_num_objects = sys.maxsize

for key, item in objects.items():
    if len(item) < min_num_objects:
        min_num_objects = len(item)
        rarest_object = key
print("The rarest object in the pictures is %s" % (rarest_object,))
print("Appears %d times" % min_num_objects)

The rarest object in the pictures is ('rectangle', 'green', 'big')
Appears 511 times


In [70]:
# take a number of samples from each object that equals the number of appearance of the rarest object

objects_balanced = {}

for key, item in objects.items():
    objects_balanced[key] = item[:min_num_objects]
    
objects = objects_balanced

# Create "1 small 2 big" and "2 big one small" scenes from balanced objects

In [71]:
# 1 small 2 big

dataset1 = {}

for shape in shapes:
    for color in colors:
        i = 0
        j = 0
        dataset1[(shape, color)] = []
        seen_pics = []
        while (i < int(len(objects[(shape, color, 'big')]) / 2) and i < len(objects[(shape, color, 'small')])):
            item1 = objects[(shape, color, 'small')][i]
            item2 = objects[(shape, color, 'big')][j]
            item3 = objects[(shape, color, 'big')][j+1]
            id1 = item1[0]
            id2 = item2[0]
            id3 = item3[0]
            # do not repeat images across single set of 3, and across the target
            if (id1 not in [id2, id3]
                and id3 not in [id2, id1] 
                and id1 not in seen_pics 
                and id2 not in seen_pics 
                and id3 not in seen_pics):
                dataset1[(shape, color)].append([item1, item2, item3])
                seen_pics.extend([id1, id2, id3])
            j += 2
            i += 1

In [72]:
print('Number of "1 small 2 big" samples per object:\n')
data_len = 0
for key, item in dataset1.items():
    print(key, len(item))
    data_len += len(item)
print(data_len)

Number of "1 small 2 big" samples per object:

('triangle', 'red') 255
('triangle', 'blue') 255
('triangle', 'yellow') 255
('triangle', 'green') 255
('triangle', 'white') 255
('circle', 'red') 255
('circle', 'blue') 255
('circle', 'yellow') 255
('circle', 'green') 255
('circle', 'white') 255
('square', 'red') 255
('square', 'blue') 255
('square', 'yellow') 255
('square', 'green') 255
('square', 'white') 255
('rectangle', 'red') 255
('rectangle', 'blue') 255
('rectangle', 'yellow') 255
('rectangle', 'green') 255
('rectangle', 'white') 255
5100


In [73]:
# 2 small 1 big

dataset2 = {}

for shape in shapes:
    for color in colors:
        i = 0
        j = 0
        dataset2[(shape, color)] = []
        seen_pics = []
        while (i < int(len(objects[(shape, color, 'small')]) / 2) and i < len(objects[(shape, color, 'big')])):
            item1 = objects[(shape, color, 'big')][i]
            item2 = objects[(shape, color, 'small')][j]
            item3 = objects[(shape, color, 'small')][j+1]
            id1 = item1[0]
            id2 = item2[0]
            id3 = item3[0]
            # do not repeat images across single set of 3, and across the target
            if (id1 not in [id2, id3]
                and id3 not in [id2, id1]
                and id1 not in seen_pics 
                and id2 not in seen_pics 
                and id3 not in seen_pics):
                dataset2[(shape, color)].append([item1, item2, item3])
                seen_pics.extend([id1, id2, id3])
            j += 2
            i += 1

In [74]:
print('Number of "1 big 2 small" samples per object:\n')
data_len = 0
for key, item in dataset2.items():
    print(key, len(item))
    data_len += len(item)
print(data_len)

Number of "1 big 2 small" samples per object:

('triangle', 'red') 255
('triangle', 'blue') 255
('triangle', 'yellow') 255
('triangle', 'green') 255
('triangle', 'white') 255
('circle', 'red') 255
('circle', 'blue') 255
('circle', 'yellow') 255
('circle', 'green') 255
('circle', 'white') 255
('square', 'red') 255
('square', 'blue') 255
('square', 'yellow') 255
('square', 'green') 255
('square', 'white') 255
('rectangle', 'red') 255
('rectangle', 'blue') 255
('rectangle', 'yellow') 255
('rectangle', 'green') 255
('rectangle', 'white') 255
5100


# Add statements with GT true/false answers

In [75]:
templates_big = [Template('There are exactly two $color $shape that are big $shape'),
                           Template('There is exactly one $color $shape that is a big $shape'),
                            Template('There are more $color $shape that are big $shape')]

templates_small = [Template('There are exactly two $color $shape that are small $shape'),
                           Template('There is exactly one $color $shape that is a small $shape'),
                            Template('There are more $color $shape that are small $shape')]

In [76]:
# 1 small 2 big

captions1 = {}
answers1 = {}

for key, items in dataset1.items():
    answers1[key] = []
    captions1[key] = []
    shape = key[0]
    color = key[1]
    for triple in items:
        
        capt_tmp = []
        
        for i, tmpl in enumerate(templates_big):
            if i % 2 == 0:
                capt_tmp.append(tmpl.substitute(shape = shape+'s', color = color))
            else:
                capt_tmp.append(tmpl.substitute(shape = shape, color = color))
                
        for i, tmpl in enumerate(templates_small):
            if i % 2 == 0:
                capt_tmp.append(tmpl.substitute(shape = shape+'s', color = color))
            else:
                capt_tmp.append(tmpl.substitute(shape = shape, color = color))
                
        captions1[key].append(capt_tmp)
                    
        answers1[key].append([True, False, True, False, True, False])

In [77]:
# 2 small 1 big

captions2 = {}
answers2 = {}

for key, items in dataset2.items():
    answers2[key] = []
    captions2[key] = []
    shape = key[0]
    color = key[1]
    for triple in items:
        
        capt_tmp = []
        
        for i, tmpl in enumerate(templates_big):
            if i % 2 == 0:
                capt_tmp.append(tmpl.substitute(shape = shape+'s', color = color))
            else:
                capt_tmp.append(tmpl.substitute(shape = shape, color = color))
                
        for i, tmpl in enumerate(templates_small):
            if i % 2 == 0:
                capt_tmp.append(tmpl.substitute(shape = shape+'s', color = color))
            else:
                capt_tmp.append(tmpl.substitute(shape = shape, color = color))
                
        captions2[key].append(capt_tmp)
        
        answers2[key].append([False, True, False, True, False, True])

In [78]:
# combine everything before saving to *.json

def get_output(dataset, captions, answers):
    output = {}
    for i, (key, pics) in enumerate(dataset.items()):
        output[i] = {}
        output[i]["target"] = {'shape': key[0], 'color': key[1]} 
        output[i]["samples"] = {}
        for sample_id, sample in enumerate(pics):
            output[i]["samples"][sample_id] = {}
            triples = []
            for img_id, target_id, target_size in sample:
                triples.append(img_id+'.png')
    #                             'image_id': img_id,
    #                             'target_id': target_id, 
    #                             'target_size': target_size
            questions = []
            for j, q in enumerate(captions[key][sample_id]):
                questions.append({})
                questions[j]['question'] = q
                questions[j]['answer'] = answers[key][sample_id][j]
            output[i]["samples"][sample_id]['questions'] = questions
            output[i]["samples"][sample_id]['images'] = triples
            
    return output

In [79]:
# 1 small 2 big
output1 = get_output(dataset1, captions1, answers1)
# 2 small 1 big
output2 = get_output(dataset2, captions2, answers2)

In [87]:
# save
with open('dataset-balanced/1small_2big_balanced/annotations/train.json', 'w', encoding='utf-8') as f:
    json.dump(output1, f, ensure_ascii=False, indent=4)

In [81]:
# save
with open('dataset-balanced/2small_1big_balanced/annotations/train.json', 'w', encoding='utf-8') as f:
    json.dump(output2, f, ensure_ascii=False, indent=4)

# Check the dataset

In [90]:
# set paths to the dataset and to the corresponding images

images_path = "malevic/data/pos1/images/train/"

with open('dataset-balanced/2small_1big_balanced/annotations/train.json') as f:
    dataset = json.load(f)

In [91]:

t = str(randrange(len(dataset)+1))
s = str(randrange(len(dataset[t]['samples'])))

img = [widgets.Image(value = open(images_path+x, 'rb').read()) for x in dataset[t]['samples'][s]['images']]

print(dataset[t]['samples'][s]['images'])
print(dataset[t]['samples'][s]['questions'])
    
hbox = HBox([img[0], img[1], img[2]])
display(hbox)

['5164.png', '13871.png', '13706.png']
[{'question': 'There are exactly two yellow triangles that are big triangles', 'answer': False}, {'question': 'There is exactly one yellow triangle that is a big triangle', 'answer': True}, {'question': 'There are more yellow triangles that are big triangles', 'answer': False}, {'question': 'There are exactly two yellow triangles that are small triangles', 'answer': True}, {'question': 'There is exactly one yellow triangle that is a small triangle', 'answer': False}, {'question': 'There are more yellow triangles that are small triangles', 'answer': True}]


# Add merged images to the dataset

In [22]:
# https://stackoverflow.com/questions/55207987/is-there-any-way-to-merge-multiple-image-into-one-single-image-in-python

class Montage(object):
    def __init__(self,initial_image):
        self.montage = initial_image
        self.x,self.y = self.montage.shape[:2]

    def append(self,image):
        image = image[:,:,:3]
        x,y = image.shape[0:2]
        new_image = cv2.resize(image,(int(y*float(self.x)/x),self.x))
        self.montage = np.hstack((self.montage,new_image))

    def save(self, filename):
        cv2.imwrite(filename, self.montage) 

In [6]:
sep = cv2.imread("separator.png")
img_id = 0

dataset_folder = 'dataset-balanced/'

for task in ('1small_2big_balanced/', '2small_1big_balanced/'):
    for split in ('test', 'val', 'train'):
        
        images_path = "malevic/data/pos1/images/" + split
        
        with open(dataset_folder + task + 'annotations/' + split + ".json") as f:
            dataset = json.load(f)
            
        for key, target_data in dataset.items():
            for i, (sample_id, sample) in enumerate(target_data['samples'].items()):
                # creating one image sample from 3 randomly shuffled examples, separated by grey area
                
                sampled_imgs = sample['images']
                
                images = [cv2.imread(images_path+'/'+x)[:,:,:3] for x in sampled_imgs]
                
                if i % 3 == 0:
                    m = Montage(images[0])
                    m.append(sep)
                    m.append(images[1])
                    m.append(sep)
                    m.append(images[2])
                elif i % 3 == 1:
                    m = Montage(images[1])
                    m.append(sep)
                    m.append(images[0])
                    m.append(sep)
                    m.append(images[2])
                else:
                    m = Montage(images[1])
                    m.append(sep)
                    m.append(images[2])
                    m.append(sep)
                    m.append(images[0])
                    
                sample['merged'] = str(img_id) + ".png"
                m.save(dataset_folder + task + "images_big_ordered/" + split + "/" + sample['merged'])
                
                img_id += 1
                
        with open(dataset_folder + task + "annotations/" + split + ".json", 'w', encoding='utf-8') as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)


# Dataset with 20K scenes

In [5]:
pos1_folder = "e:/Study/Thesis/code/malevic/data/pos1/"

In [6]:
# aggregate pics into groups by split+shape+color

aggregated_pics = {}

for split in ['test', 'val', 'train']:
    
    aggregated_pics[split] = {}
    
    with open(pos1_folder + "dataset/" + split + ".json") as f:
        dataset_split = json.load(f)
        
    for info in dataset_split['questions']:

        question_tokens = info['question'].split()
            
        shape = question_tokens[2]
        color = question_tokens[1]
        size = question_tokens[5]
        
        if info['answer'] == 'no':
            if size == 'small':
                size = 'big'
            elif size == 'big':
                size = 'small'
        
        target = (shape, color, size)
        
        if target in aggregated_pics[split]:
            aggregated_pics[split][target].append(info['image_filename'])
        else:
            aggregated_pics[split][target] = [info['image_filename']]
    

In [8]:
# build scenes

dataset_pics = {}

for split in ['test', 'val', 'train']:
    
    dataset_pics[split] = {'1small2big':{}, '1big2small':{}}
    
    for target, images in aggregated_pics[split].items():       
        if target[2] == 'small':
            size = 'big'
            task = '1small2big'
        else:
            size = 'small'
            task = '1big2small'
        
        add_target = (target[0], target[1], size)
        
        for img1 in images:
            img2 = random.choice(aggregated_pics[split][add_target])
            img3 = random.choice(aggregated_pics[split][add_target])
            
            if ((target[0], target[1]) in dataset_pics[split][task]):
                dataset_pics[split][task][(target[0], target[1])].append([img1, img2, img3])
            else:
                dataset_pics[split][task][(target[0], target[1])] = [[img1, img2, img3]]

In [39]:
len(dataset_pics['train']['1small2big'][('circle','green')])

400

In [11]:
# statistics check

for split, task_data in dataset_pics.items():
    print('============')
    print("Split:", split)
    print('============')
    for task, main_data in task_data.items():
        i = task.find('1')
        j = task.find('2')
        print("Size:", task[i+1:j])
        for target, pic_list in main_data.items():
            print(target, ': ', len(pic_list))

Split: test
Size: small
('circle', 'green') :  50
('triangle', 'white') :  50
('rectangle', 'white') :  50
('circle', 'red') :  50
('square', 'yellow') :  50
('circle', 'yellow') :  50
('square', 'blue') :  50
('triangle', 'red') :  50
('circle', 'white') :  50
('circle', 'blue') :  50
('rectangle', 'green') :  50
('square', 'white') :  50
('rectangle', 'yellow') :  50
('square', 'green') :  50
('rectangle', 'red') :  50
('triangle', 'yellow') :  50
('square', 'red') :  50
('triangle', 'blue') :  50
('triangle', 'green') :  50
('rectangle', 'blue') :  50
Size: big
('square', 'green') :  50
('rectangle', 'red') :  50
('square', 'red') :  50
('triangle', 'white') :  50
('square', 'white') :  50
('circle', 'blue') :  50
('circle', 'white') :  50
('triangle', 'yellow') :  50
('rectangle', 'yellow') :  50
('rectangle', 'green') :  50
('circle', 'yellow') :  50
('circle', 'green') :  50
('rectangle', 'blue') :  50
('triangle', 'green') :  50
('square', 'blue') :  50
('triangle', 'blue') :  5

In [12]:
# sentences templates

templates_big = [Template('There are exactly two $color $shape that are big $shape'),
                           Template('There is exactly one $color $shape that is a big $shape'),
                            Template('There are more $color $shape that are big $shape')]

templates_small = [Template('There are exactly two $color $shape that are small $shape'),
                           Template('There is exactly one $color $shape that is a small $shape'),
                            Template('There are more $color $shape that are small $shape')]

In [41]:
# add sentences and gt labels

questions = {}

for split, task_data in dataset_pics.items():
    
    questions[split] = []
    
    question_index = 0
    image_index = 0
    
    for task, main_data in task_data.items():
        for target, pic_list in main_data.items():
            
            shape = target[0]
            color = target[1]           
            
            for img_triple in pic_list:
                
                questions_curr = []
                
                for i, tmpl in enumerate(templates_big):
                    if i % 2 == 0:
                        q = tmpl.substitute(shape = shape+'s', color = color)
                        questions_curr.append(q)
                    else:
                        q = tmpl.substitute(shape = shape, color = color)
                        questions_curr.append(q)

                for i, tmpl in enumerate(templates_small):
                    if i % 2 == 0:
                        q = tmpl.substitute(shape = shape+'s', color = color)
                        questions_curr.append(q)
                    else:
                        q = tmpl.substitute(shape = shape, color = color)
                        questions_curr.append(q)
                if task == '1big2small':
                    answers_curr = [False, True, False, True, False, True]
                elif task == '1small2big':
                    answers_curr = [True, False, True, False, True, False]

                for i, question in enumerate(questions_curr):
                    q_dict = {'question_index': question_index,
                              'image_index': image_index,
                              'images': img_triple,
                              'question': question,
                              'answer': answers_curr[i]
                             }

                    questions[split].append(q_dict)
                    question_index += 1

                image_index += 1
            

In [45]:
questions['test'][-10:-1]

[{'question_index': 11990,
  'image_index': 1998,
  'images': ['1848.png', '265.png', '457.png'],
  'question': 'There are more red triangles that are big triangles',
  'answer': False},
 {'question_index': 11991,
  'image_index': 1998,
  'images': ['1848.png', '265.png', '457.png'],
  'question': 'There are exactly two red triangles that are small triangles',
  'answer': True},
 {'question_index': 11992,
  'image_index': 1998,
  'images': ['1848.png', '265.png', '457.png'],
  'question': 'There is exactly one red triangle that is a small triangle',
  'answer': False},
 {'question_index': 11993,
  'image_index': 1998,
  'images': ['1848.png', '265.png', '457.png'],
  'question': 'There are more red triangles that are small triangles',
  'answer': True},
 {'question_index': 11994,
  'image_index': 1999,
  'images': ['1849.png', '271.png', '472.png'],
  'question': 'There are exactly two red triangles that are big triangles',
  'answer': False},
 {'question_index': 11995,
  'image_index'

In [70]:
images_path = "malevic/data/pos1/images/test/"

random_scene = random.choice(dataset_pics['test']['1big2small'][('circle',
    'green')])

img = [widgets.Image(value = open(images_path+x, 'rb').read()) for x in ['25.png', '1444.png', '878.png']]
    
hbox = HBox([img[0], img[1], img[2]])
display(hbox)

In [ ]:
# create nlvr-like images

sep = cv2.imread("separator.png")

for split, questions_list in questions.items():
    
    for i, q in enumerate(questions_list):
        i = 0
        images_path = "malevic/data/pos1/images/" + split + "/"

        img_id = q['image_index']
        q['image_filename'] = str(img_id) + ".png"

        sampled_imgs = q['images']

        if i % 6 == 0:
            images = [cv2.imread(images_path+'/'+x)[:,:,:3] for x in sampled_imgs]

            if img_id % 3 == 0:
                m = Montage(images[0])
                m.append(sep)
                m.append(images[1])
                m.append(sep)
                m.append(images[2])
            elif img_id % 3 == 1:
                m = Montage(images[1])
                m.append(sep)
                m.append(images[0])
                m.append(sep)
                m.append(images[2])
            else:
                m = Montage(images[1])
                m.append(sep)
                m.append(images[2])
                m.append(sep)
                m.append(images[0])

            m.save("dataset-balanced/images_big_20K_ordered/" + split + "/" + q['image_filename'])

In [15]:
len(questions['train'])

96000

In [46]:
for split, questions_list in questions.items():
    for q in questions_list:
        if q['question_index'] % 6 == 0:
            if q['answer'] == True:
                curr_task = "1small2big"
            elif q['answer'] == False:
                curr_task = "1big2small"
        q['task'] = curr_task
    

In [47]:
for split in ['train', 'test', 'val']:
    with open("dataset-balanced/questions/" + split + "_questions.json", "w") as f:
        json.dump(questions[split], f)

In [48]:
for split in ['train', 'test', 'val']:
    with open("dataset-balanced/questions/" + split + "_questions.json") as f:
        dataset = json.load(f)